In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
from scipy.special import psi
from datetime import datetime as dt

In [73]:
%%time
%matplotlib

! mkdir -p figure

N=100
T=40
K=4

X1=np.random.multivariate_normal([1,2],[[1,0],[0,1]],20)
X2=np.random.multivariate_normal([2,-2],[[1,0],[0,1]],30)
X3=np.random.multivariate_normal([-1,-1],[[1,0],[0,1]],50)
X=np.concatenate((X1,X2,X3),axis=0)

phi=[1 for k in range(K)]
mu=[np.zeros(2)+np.random.rand(2)/10 for k in range(K)]
tau=[1 for k in range(K)]

for t in range(T):
    plt.figure(figsize=(10,10))
    plt.xlim(-5,5)
    plt.ylim(-5,5)

    plt.scatter(X1[:,0],X1[:,1])
    plt.scatter(X2[:,0],X2[:,1])
    plt.scatter(X3[:,0],X3[:,1])

    lin=np.linspace(-5,5,100)
    x0,x1=np.meshgrid(lin,lin)
    p_predict=0
    for k in range(K):
        p_predict_k=1
        p_predict_k*=(phi[k]/(N+K))*(1/(2*np.pi))*(tau[k]/(1+tau[k]))
        p_predict_k*=np.exp(-((x0-mu[k][0])**2+(x1-mu[k][1])**2)*tau[k]/(2*(1+tau[k])))
        p_predict+=p_predict_k
    plt.contour(x0,x1,p_predict)
    plt.title("N="+str(N)+", iter="+str(t))
    plt.savefig("figure/"+str(t).zfill(3)+".png")
    
    L=[]
    for i in range(N):
        Li=[]
        for k in range(K):
            Lik=psi(phi[k])-psi(sum(phi))-1/tau[k]
            Lik=Lik-((X[i][0]-mu[k][0])**2+(X[i][1]-mu[k][1])**2)/2
            Li.append(Lik)
        L.append(Li)
    Y=[]
    for i in range(N):
        Yi=[]
        for k in range(K):
            Yik=np.exp(L[i][k])/sum([np.exp(L[i][k]) for k in range(K)])
            Yi.append(Yik)
        Y.append(Yi)
    for k in range(K):
        phi[k]=phi[k]+sum([Y[i][k] for i in range(N)])
        mu[k]=(tau[k]*mu[k]+sum([X[i]*Y[i][k] for i in range(N)]))/(tau[k]+sum([Y[i][k] for i in range(N)]))
        tau[k]=tau[k]+sum([Y[i][k] for i in range(N)])

files = sorted(glob.glob('figure/*.png'))
images = list(map(lambda file: Image.open(file), files))
now=dt.now().strftime("%Y%m%d%H%M%S")
images[0].save(now+'.gif', save_all=True, append_images=images[1:], duration=3, loop=1)
print("complete")
!rm figure/*.png

Using matplotlib backend: agg


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


complete
CPU times: user 14.8 s, sys: 4.44 s, total: 19.2 s
Wall time: 14.9 s
